### Pythonで（統計的な）回帰分析を試す

In [7]:
import pandas as pd
import seaborn as sns

In [8]:
df = sns.load_dataset("tips")
df = pd.get_dummies(df)
df.head()

,total_bill,tip,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,1.01,2,0,1,0,1,0,0,0,1,0,1
1,10.34,1.66,3,1,0,0,1,0,0,0,1,0,1
2,21.01,3.50,3,1,0,0,1,0,0,0,1,0,1
3,23.68,3.31,2,1,0,0,1,0,0,0,1,0,1
4,24.59,3.61,4,0,1,0,1,0,0,0,1,0,1


In [9]:
#多重共線性排除
df.drop(["sex_Female", "smoker_No", "day_Thur", "time_Lunch"], axis=1, inplace=True)

In [10]:
import statsmodels.api as sm

X = df.drop('tip', 1)
X = sm.add_constant(X)#バイアス項を追加
y = df['tip']
model = sm.OLS(y, X)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    tip   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     26.06
Date:                Thu, 24 Jan 2019   Prob (F-statistic):           1.20e-28
Time:                        22:58:08   Log-Likelihood:                -347.48
No. Observations:                 244   AIC:                             713.0
Df Residuals:                     235   BIC:                             744.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.7097      0.232      3.054      0.003       0.252       1.167
total_bill      0.0945      0.010      9.841      0.000       0.076       0.113
size            0.1760      0.090      1.966      0.051      -0.000       0.352
sex_Male       -0.0324      0.142     -0.229      0.819      -0.311       0.247
smoker_Yes     -0.0864      0.147     -0.589      0.556      -0.375       0.202
day_Fri         0.1623      0.393      0.412      0.680      -0.613       0.937
day_Sat         0.0408      0.471      0.087      0.931      -0.886       0.968
day_Sun         0.1368      0.472      0.290      0.772      -0.793       1.066
time_Dinner    -0.0681      0.445     -0.153      0.878      -0.944       0.808
==============================================================================
Omnibus:                       27.860   Durbin-Watson:                   2.096
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               52.555
Skew:                           0.607   Prob(JB):                     3.87e-12
Kurtosis:                       4.923   Cond. No.                         281.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
import statsmodels.formula.api as smf

mod = smf.ols(formula="tip ~  + total_bill + size", data=df)
result = mod.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    tip   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     105.9
Date:                Thu, 24 Jan 2019   Prob (F-statistic):           9.67e-34
Time:                        22:58:08   Log-Likelihood:                -347.99
No. Observations:                 244   AIC:                             702.0
Df Residuals:                     241   BIC:                             712.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6689      0.194      3.455      0.001       0.288       1.050
total_bill     0.0927      0.009     10.172      0.000       0.075       0.111
size           0.1926      0.085      2.258      0.025       0.025       0.361
==============================================================================
Omnibus:                       24.753   Durbin-Watson:                   2.100
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.169
Skew:                           0.545   Prob(JB):                     9.43e-11
Kurtosis:                       4.831   Cond. No.                         67.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
#vifを計算する
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
X = df.drop(["tip"], 1)
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif

,VIF Factor,features
0,10.003691,total_bill
1,9.869349,size
2,2.824887,sex_Male
3,1.823828,smoker_Yes
4,2.720204,day_Fri
5,18.194395,day_Sat
6,16.039843,day_Sun
7,33.152732,time_Dinner


pythonには、ステップワイズ法はない模様。